In [ ]:
#| default_exp clinical_llms

# Extracting clinical data using LLMs
> What is this nb about?

In [ ]:
#| hide
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

#### meta-llama/Llama-2-7b-hf

In [ ]:
mn = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)
tokr = AutoTokenizer.from_pretrained(mn)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
prompt = "Extract the calcification status or category from the following radiology report: "

In [ ]:
report = """
FINDINGS:
There are scattered calcifications in the right breast, which are coarse and heterogeneous in morphology. These calcifications are categorized as BI-RADS 4, suspicious for malignancy.

No suspicious masses, architectural distortion, or skin thickening are identified in either breast.

IMPRESSION:
BI-RADS 4: Suspicious calcifications in the right breast. Biopsy is recommended for further evaluation.
"""

In [ ]:
ques = prompt + report

In [ ]:
toks = tokr(ques, return_tensors="pt")

In [ ]:
tokr.batch_decode(toks['input_ids'])

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
tokr.batch_decode(res)

#### TheBloke/Llama-2-13B-GPTQ

In [ ]:
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 50)

In [ ]:
#### TheBloke/OpenOrca-Platypus2-13B-GPTQ

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques), 150)

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
sb_sys = "### System:\nYou are an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
prompt = "Extract the calcification status or category from the following radiology report: "

In [ ]:
report = """
FINDINGS:
There are scattered calcifications in the right breast, which are coarse and heterogeneous in morphology. These calcifications are categorized as BI-RADS 4, suspicious for malignancy.

No suspicious masses, architectural distortion, or skin thickening are identified in either breast.

IMPRESSION:
BI-RADS 4: Suspicious calcifications in the right breast. Biopsy is recommended for further evaluation.
"""

ques = prompt + report

gen(mk_prompt(ques), 150)

#### RAG?

In [ ]:
#| hide
webpage = """
Breast calcifications
Last revised by Mostafa Elfeky on 10 Jul 2023
 
Citation, DOI, disclosures and article data
Breast calcifications are deposits of calcium salts in the breast, which are radio-opaque on mammography. The majority are benign, but they can be associated with cancer. The ability to diagnose and appropriately manage the significant microcalcifications and differentiate them from innocuous findings is part of the art and science of breast imaging.

Epidemiology
Calcifications are extremely common, present in ~85% of mammograms 8. Their frequency increases with age. 

In a screening mammography program, the rate of recall because of calcifications was 1.7% and of these, 19% resulted in a cancer diagnosis 13. In the digital mammography era, about one-sixth of all recalls are for calcifications 15.

Calcification-specific cancer detection rates in the era of digital mammography range from 1.9 to 3.2 in 1000 screened 13,14. A third of breast cancers show calcifications as the only mammographically suspicious feature 13. Up to 50% of breast cancers can be associated with calcification while 15-30% of calcifications biopsied for various reasons tend to be malignant in asymptomatic patients 10.

Pathology
Aetiology
Of the breast cancers detected on mammography due to calcifications, about two-thirds represent ductal carcinoma in situ and the remainder are invasive ductal carcinoma 13,14. In contrast, the causes of benign calcifications are broad and covered in detail in other sections (Radiographic features and Differential diagnosis).

Microscopic appearance
Calcifications typically represent calcium phosphate or calcium oxalate salt deposition, with the latter more likely to be benign 19. The former appears purple on haematoxylin and eosin stain, while the latter requires the use of polarised light to assess birefringence.

Radiographic features
The BI-RADS lexicon prescribes descriptors for typically benign calcifications as well as suspicious morphologic terms and descriptors of calcification distribution 17. See also the separate article on breast calcifications: an approach.

Typically benign calcifications
skin: lucent centred, usually tightly grouped, and can be localised on tangential views if the appearance is atypical; formed within the dermis/epidermis
vascular: parallel tram tracks or discontinuous linear appearance at the margins of a tubular structure; formed in blood vessel walls
coarse or "popcorn-like": large (>2-3 mm), sometimes confluent or associated with a circumscribed mass; formed in involuting fibroadenomas
large rod-like: large tubules in a radiating and sometimes branching distribution orientated towards the nipple; with the largest in a breast measuring mean 4 mm long and 0.6 mm wide, usually bilateral, usually intraductal but sometimes periductal, usually in older women (>60 years) 18; formed within ductal ectasia
round, including punctate: circular, usually <1 mm (<0.5 mm if punctate); formed in acini of terminal ductal lobular unit
rim: "eggshell", lucent centred; formed in the walls of oil cysts (fat necrosis) or simple cysts
dystrophic: irregular, usually >1 mm; formed after radiation, trauma, or surgery
milk of calcium: "tea cups" curvilinear or linear level on the 90° lateral (ML/LM) view but a smudge on the CC view; formed as sediment within cysts
suture: linear or tubular calcifications; formed on loops and knots of surgical sutures
Suspicious morphology of calcifications
The following particle shapes carry increasing suspicion for malignancy, in order 17,18:

coarse heterogeneous: irregular, generally 0.5-1 mm
amorphous: indistinct and/or small ("powdery", "cloud", or "cottony"), such that another specific shape cannot be determined
fine pleomorphic: variable shape ("shards of glass" or "crushed stone"), generally <0.5 mm
fine linear or fine-linear branching: thin (<0.5 mm), linear, branching or irregularly arranged ("casting")
See also the separate article on suspicious breast calcifications. These descriptors and their associated probability of malignancy can be further modified by their distribution.

Distribution
diffuse: scattered randomly throughout the breast; almost always benign
regional: scattered in a larger volume (>2 cm in greatest linear dimension) of breast tissue and not in the expected ductal distribution; malignancy is less likely but the overall evaluation also depends on the morphology
grouped: a cluster of at least 5 calcifications within 1 cm from each other, in an area at most 2 cm in greatest linear dimension; more likely to be malignant with smaller calcific particle sizes (<1 mm) and suspicious shape
linear: calcifications arrayed in a line suggestive of deposition along ducts; suspicious if there is not a characteristically benign appearance such as vascular or large rod-like calcifications)
segmental: calcium deposits in ducts and branches of a segment or lobe; suspicious if not characteristically benign such as large rod-like
Radiology report
Calcifications that are typically benign do not need to be reported (BI-RADS 1), so that the referrer or patient reading the report would not be confused that they could be anything other than definitely benign, but they may be reported (BI-RADS 2) if there is concern that other observers viewing the images might not interpret them as benign 17.

Round calcifications are benign (BI-RADS 2) if diffuse or stable for several years. Grouped punctate calcifications in isolation and without prior examinations for comparison are probably benign if a full diagnostic evaluation has been performed (BI-RADS 3). Punctate calcifications that are new, increased, or linear or segmental in distribution are suspicious (BI-RADS 4).

Amorphous calcifications are suspicious (BI-RADS 4) unless they are bilateral and diffuse and/or stable for many years. A single group of coarse heterogeneous, amorphous, or fine pleomorphic calcifications are moderately suspicious (BI-RADS 4B), whereas fine linear or fine-linear branching calcifications are highly suspicious (BI-RADS 4C) 17. A linear or segmental distribution increases the level of suspicion for calcifications with suspicious morphologies.

A combination of several suspicious descriptors/findings can be highly suggestive of malignancy (BI-RADS 5), a category which is used if a nonmalignant percutaneous biopsy result would be discordant and prompt a repeat biopsy (usually surgical).

Differential diagnosis
Many entities cause breast calcifications but the primary distinction of clinical relevance is between benign and malignant causes. The classic benign aetiologies are noted above ("Typically benign calcifications"). As noted above, malignant calcifications usually represent either ductal carcinoma in situ or invasive ductal carcinoma.

Artifactual mimics of calcification include antiperspirants, powders, soaps, and tattoos.
"""

In [ ]:
context = f"{report}\n{webpage[:700]}"

In [ ]:
ques = "Based on the information provided, what type of microcalcifications are present in the report? What does this likely indicate?"

In [ ]:
ques_ctx = f"""Answer the question with the help of the provided context. ## Context {context} ## Question {ques}"""

In [ ]:
res = gen(mk_prompt(ques_ctx), 300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])